# LangChain + Qdrant Conversational RAG

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrypdamiba/qdrant-etl-cookbook/blob/main/notebooks/agents/langchain_qdrant.ipynb)

Build a conversational RAG chain with LangChain, using Qdrant as the vector store and OpenAI for generation.

**Requirements:** Set `OPENAI_API_KEY` environment variable.

In [ ]:
!pip install -q langchain langchain-openai langchain-qdrant qdrant-client

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import QdrantVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from qdrant_client import QdrantClient

In [ ]:
client = QdrantClient(":memory:")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# Create documents
docs = [
    Document(page_content="Qdrant stores vectors with associated payloads. Payloads can contain any JSON data and can be indexed for fast filtering.", metadata={"source": "docs"}),
    Document(page_content="HNSW index parameters: m controls the number of bi-directional links, ef_construct controls index build quality. Higher values mean better recall but more memory.", metadata={"source": "docs"}),
    Document(page_content="Scalar quantization converts float32 to int8, reducing memory by 4x. Use oversampling=2.0 and rescore=true for minimal accuracy loss.", metadata={"source": "docs"}),
    Document(page_content="Create payload indexes on fields you filter by. Supported types: keyword (exact match), integer (range), float (range), geo (radius), text (full-text).", metadata={"source": "docs"}),
    Document(page_content="For multi-tenancy, use a tenant_id payload field with a keyword index. All searches should filter by tenant_id to ensure data isolation.", metadata={"source": "docs"}),
]

# Build Qdrant vector store
vector_store = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    location=":memory:",
    collection_name="langchain_demo",
)
print(f"Indexed {len(docs)} documents")

In [ ]:
# Build retrieval chain
llm = ChatOpenAI(model="gpt-4o-mini")
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using the provided context. If the context doesn't have the answer, say so.\n\nContext:\n{context}"),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# Ask questions
response = rag_chain.invoke({"input": "How do I optimize memory usage in Qdrant?"})
print("Answer:", response["answer"])
print(f"\nUsed {len(response['context'])} source documents")

In [ ]:
response = rag_chain.invoke({"input": "How should I set up multi-tenancy?"})
print("Answer:", response["answer"])